In [27]:
import cv2
import numpy as np
from scipy.signal import convolve2d
from scipy.ndimage.filters import correlate 
import matplotlib.pyplot as plt
import matplotlib.image as pltimg 
import math

In [28]:
def load_image(path, mode=cv2.IMREAD_GRAYSCALE):
    '''
    :param path: image path
    :param mode: load mode, cv2.IMREAD_GRAYSCALE, cv2.IMREAD_COLOR
    :return: loaded img
    '''
    img = cv2.imread(path,mode)
    return img


In [29]:

def normalize(img):
    ''' Function to normalize an input array to 0-1 '''
    img_min = img.min()
    img_max = img.max()
    return (img - img_min) / (img_max - img_min)

def generate_gaussian_kernel(rows, cols, sigma, highPass=True):
    '''
    :param rows: rows of gaussian kernel
    :param cols: cols of gaussian kernel
    :param sigma: sigma
    :param highPass: if this kernel is in highpass mode
    :return: 
    '''
    center_row = int(rows/2) + 1 if rows % 2 == 1 else int(rows/2)
    center_col = int(cols/2) + 1 if cols % 2 == 1 else int(cols/2)
    def gaussian_at_pos(i,j):  
        coefficient = math.exp(-1.0 * ((i - center_row)**2 + (j - center_col)**2) / (2 * sigma**2))
        return 1 - coefficient if highPass else coefficient
    
    return np.array([[gaussian_at_pos(i, j) for j in range(cols)] for i in range(rows)])



In [30]:
def generated_hybrid_image(image1, image2, kernel_size1, sigma1, kernel_size2, sigma2):
    '''
    :param image1: high_frequency_image
    :param image2: low_frequency
    :param sigma1: 
    :param sigma2: 
    :return: 
    '''
    high_pass_filter = generate_gaussian_kernel(kernel_size1, kernel_size1, sigma1)
    high_pass = image1 - correlate(image1, high_pass_filter)
    
    low_pass_filter = generate_gaussian_kernel(kernel_size2, kernel_size2, sigma2)
    low_pass = correlate(image2, low_pass_filter)
    
    hybrid_img = (low_pass + high_pass)
    return hybrid_img

In [31]:
def generate_and_show_image(img_path1, img_path2, kernel_size1, sigma1, kernelsize_2,
                            sigma2, saved_name):
    high_frequency = load_image(PATH_C_1)
    low_frequency = load_image(PATH_C_2)
    high_frequency = high_frequency.astype(np.single)/255 
    low_frequency = low_frequency.astype(np.single)/255
    # print(high_frequency.shape)
    # print(low_frequency.shape)
    
    hybrid_img_C = generated_hybrid_image(high_frequency, low_frequency, kernel_size1, sigma1, kernel_size2, sigma2)
    
    ''' Visualize and save outputs '''
    plt.figure(1)
    plt.imshow(normalize(low_frequency), cmap='gray')
    plt.figure(2)
    plt.imshow(normalize(high_frequency+0.5), cmap='gray')
    plt.figure(3)
    plt.imshow(normalize(hybrid_img_C), cmap='gray')
    plt.show()
    # print(hybrid_img_C.dtype)
    cv2.imwrite('hybrid_c.png', 255 * normalize(hybrid_img_C)) 
   


In [32]:
if __name__ == '__main__':
    
    # load image
    PATH_C_1 = "data\\part1\\c1.jpg"
    PATH_C_2 = "data\\part1\\c2.jpg"
    sigma1 = 10
    kernel_size1 = 5
    sigma2 = 10
    kernel_size2 = 5
    
    generate_and_show_image(PATH_C_1, PATH_C_2, kernel_size1,sigma1,
                            kernel_size2,sigma2,"hybrid_c.png")
    